<a href="https://colab.research.google.com/github/monkey21254/Project/blob/main/myCutMix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [23]:
# git clone
!git clone https://github.com/Britefury/cutmix-semisup-seg

Cloning into 'cutmix-semisup-seg'...
remote: Enumerating objects: 103, done.
remote: Counting objects: 100% (103/103), done.
remote: Compressing objects: 100% (76/76), done.
remote: Total 103 (delta 36), reused 85 (delta 21), pack-reused 0
Receiving objects: 100% (103/103), 3.58 MiB | 6.58 MiB/s, done.
Resolving deltas: 100% (36/36), done.


In [2]:
# 파이썬 버전 확인
import sys
sys.version_info

sys.version_info(major=3, minor=6, micro=9, releaselevel='final', serial=0)

In [3]:
# 디스크 용량
!df -h

Filesystem      Size  Used Avail Use% Mounted on
overlay          69G   33G   37G  48% /
tmpfs            64M     0   64M   0% /dev
tmpfs           6.4G     0  6.4G   0% /sys/fs/cgroup
shm             5.8G     0  5.8G   0% /dev/shm
tmpfs           6.4G   20K  6.4G   1% /var/colab
/dev/sda1        75G   34G   41G  46% /opt/bin
tmpfs           6.4G     0  6.4G   0% /proc/acpi
tmpfs           6.4G     0  6.4G   0% /proc/scsi
tmpfs           6.4G     0  6.4G   0% /sys/firmware
drive            15G  7.5G  7.6G  50% /content/drive


In [4]:
# 메모리 정보
!cat /proc/meminfo

MemTotal:       13333596 kB
MemFree:        10524888 kB
MemAvailable:   12439544 kB
Buffers:           82036 kB
Cached:          1981988 kB
SwapCached:            0 kB
Active:           981948 kB
Inactive:        1493536 kB
Active(anon):     386668 kB
Inactive(anon):      356 kB
Active(file):     595280 kB
Inactive(file):  1493180 kB
Unevictable:           0 kB
Mlocked:               0 kB
SwapTotal:             0 kB
SwapFree:              0 kB
Dirty:              7228 kB
Writeback:             0 kB
AnonPages:        411560 kB
Mapped:           241636 kB
Shmem:              1008 kB
Slab:             179656 kB
SReclaimable:     133908 kB
SUnreclaim:        45748 kB
KernelStack:        5056 kB
PageTables:         6048 kB
NFS_Unstable:          0 kB
Bounce:                0 kB
WritebackTmp:          0 kB
CommitLimit:     6666796 kB
Committed_AS:    3359476 kB
VmallocTotal:   34359738367 kB
VmallocUsed:           0 kB
VmallocChunk:          0 kB
Percpu:             1048 kB
AnonHugePages:   

In [5]:
# GPU 사용가능여부 확인
!nvidia-smi

Tue Feb 16 06:30:36 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [6]:
# %cd ..
%cd cutmix-semisup-seg
# %pip install -qr requirements.txt

/content/cutmix-semisup-seg


In [8]:
import torch
from IPython.display import Image, clear_output

clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

Setup complete. Using torch 1.7.0+cu101 _CudaDeviceProperties(name='Tesla T4', major=7, minor=5, total_memory=15109MB, multi_processor_count=40)


In [9]:
%pip install git+https://github.com/Britefury/semantic-segmentation-pytorch.git@logits-from-models

  Cloning https://github.com/Britefury/semantic-segmentation-pytorch.git (to revision logits-from-models) to /tmp/pip-req-build-z5hsu4rg
  Running command git clone -q https://github.com/Britefury/semantic-segmentation-pytorch.git /tmp/pip-req-build-z5hsu4rg
  Running command git checkout -b logits-from-models --track origin/logits-from-models
  Switched to a new branch 'logits-from-models'
  Branch 'logits-from-models' set up to track remote branch 'logits-from-models' from 'origin'.
  Created wheel for mit-semseg: filename=mit_semseg-1.0.0-cp36-none-any.whl size=46732 sha256=0cdda3c39856a2ce88cd9187b56338e2cb87473d2252be145c9998291cd311e3
  Stored in directory: /tmp/pip-ephem-wheel-cache-ewf_sku5/wheels/02/2a/b0/5fbf7ecd42ac24c2a373706abe9a1c55721120ec3b094e1fa9
Successfully built mit-semseg


In [10]:
import os
os.getcwd()

'/content/cutmix-semisup-seg'

In [11]:
def make_dir(name):
  if not os.path.isdir(name):
    os.mkdir(name)
  else:
    print("Already exists.")

#### cfg 파일 작성

In [28]:
make_path = ['[paths]\n',
'camvid=/datasets/camvid/CamVidData.zip\n',
'cityscapes=/datasets/cityscapes\cityscapes_segmentation.zip\n',
'isic2017=/datasets/isic2017/isic2017_segmentation_248x248.zip\n',
'pascal_voc=/content/cutmix-semisup-seg/VOCdevkit/VOC2012']
# 'pascal_voc=/datasets/pascal_voc2012/VOCdevkit/VOC2012'에서 수정

In [29]:
with open("semantic_segmentation.cfg", 'w') as f:
    f.writelines(make_path)

#### tar 파일 압축풀기

In [ ]:
# tar 파일 압축 풀기
!tar -xvf /content/drive/MyDrive/temp/VOCtrainval_11-May-2012.tar -C .

In [ ]:
# zip 파일 압축 풀기
!unzip /content/drive/MyDrive/temp/SegmentationClassAug.zip -d /content/cutmix-semisup-seg/VOCdevkit/VOC2012

In [31]:
!python download_pascal_aug_names.py

In [ ]:
!sh run_pascal_aug_deeplab3plus_experiments.sh 1000 5

[NO dnnlib] logging to results/train_seg_semisup_mask_mt/log_pascalaug_deeplab3plusi_lr1e-5_sup_5_split0.txt
Loaded data
Built network
Settings:
arch=resnet101_deeplabv3plus_imagenet, aug_hflip=True, aug_hvflip=False, aug_max_scale=1.0, aug_rot_mag=0.0, aug_scale_hung=True, aug_scale_non_uniform=False, aug_vflip=False, batch_size=10, bin_fill_holes=False, boxmask_by_size=False, boxmask_fixed_aspect_ratio=False, boxmask_n_boxes=1, boxmask_no_invert=False, boxmask_outside_bounds=False, conf_per_pixel=False, conf_thresh=0.97, cons_loss_fn=var, cons_weight=0.0, crop_size=321,321, dataset=pascal_aug, freeze_bn=True, iters_per_epoch=1000, learning_rate=1e-05, lr_poly_power=0.9, lr_sched=none, lr_step_epochs=, lr_step_gamma=0.1, mask_mode=mix, mask_prop_range=0.5, model=mean_teacher, n_sup=1000, n_unsup=-1, n_val=-1, num_epochs=40, num_workers=4, opt_type=adam, rampup=-1, save_model=False, save_preds=False, sgd_momentum=0.9, sgd_nesterov=False, sgd_weight_decay=0.0005, split_path=./data/split

In [30]:
0# !rm -r ./results